In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:
def add_ad_to_car_dict(ad):
    try:
        car_url = ad
        response = requests.get(car_url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
    except:
        return
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 200000
for ad in all_ads[200000:230000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue
df_9 = pd.DataFrame.from_dict(car_dict, orient='index')
df_9.to_csv('Final_df_9.csv')

200000
200001
200002
200003
200004
200005
200006
200007
200008
200009
Completed 200010 of total 269381
200010
200011
200012
200013
200014
200015
200016
200017
200018
200019
Completed 200020 of total 269381
200020
200021
200022
200023
200024
200025
200026
200027
200028
200029
Completed 200030 of total 269381
200030
200031
200032
200033
200034
200035
200036
200037
200038
200039
Completed 200040 of total 269381
200040
200041
200042
200043
200044
200045
200046
200047
200048
200049
Completed 200050 of total 269381
200050
200051
200052
200053
200054
200055
200056
200057
200058
200059
Completed 200060 of total 269381
200060
200061
200062
200063
200064
200065
200066
200067
200068
200069
Completed 200070 of total 269381
200070
200071
200072
200073
200074
200075
200076
200077
200078
200079
Completed 200080 of total 269381
200080
200081
200082
200083
200084
200085
200086
200087
200088
200089
Completed 200090 of total 269381
200090
200091
200092
200093
200094
200095
200096
200097
200098
200099
Com

200798
200799
Completed 200800 of total 269381
200800
200801
200802
200803
200804
200805
200806
200807
200808
200809
Completed 200810 of total 269381
200810
200811
200812
200813
200814
200815
200816
200817
200818
200819
Completed 200820 of total 269381
200820
200821
200822
200823
200824
200825
200826
200827
200828
200829
Completed 200830 of total 269381
200830
200831
200832
200833
200834
200835
200836
200837
200838
200839
Completed 200840 of total 269381
200840
200841
200842
200843
200844
200845
200846
200847
200848
200849
Completed 200850 of total 269381
200850
200851
200852
200853
200854
200855
200856
200857
200858
200859
Completed 200860 of total 269381
200860
200861
200862
200863
200864
200865
200866
200867
200868
200869
Completed 200870 of total 269381
200870
200871
200872
200873
200874
200875
200876
200877
200878
200879
Completed 200880 of total 269381
200880
200881
200882
200883
200884
200885
200886
200887
200888
200889
Completed 200890 of total 269381
200890
200891
200892
20089

201592
201593
201594
201595
201596
201597
201598
201599
Completed 201600 of total 269381
201600
201601
201602
201603
201604
201605
201606
201607
201608
201609
Completed 201610 of total 269381
201610
201611
201612
201613
201614
201615
201616
201617
201618
201619
Completed 201620 of total 269381
201620
201621
201622
201623
201624
201625
201626
201627
201628
201629
Completed 201630 of total 269381
201630
201631
201632
201633
201634
201635
201636
201637
201638
201639
Completed 201640 of total 269381
201640
201641
201642
201643
201644
201645
201646
201647
201648
201649
Completed 201650 of total 269381
201650
201651
201652
201653
201654
201655
201656
201657
201658
201659
Completed 201660 of total 269381
201660
201661
201662
201663
201664
201665
201666
201667
201668
201669
Completed 201670 of total 269381
201670
201671
201672
201673
201674
201675
201676
201677
201678
201679
Completed 201680 of total 269381
201680
201681
201682
201683
201684
201685
201686
201687
201688
201689
Completed 201690 

Completed 202390 of total 269381
202390
202391
202392
202393
202394
202395
202396
202397
202398
202399
Completed 202400 of total 269381
202400
202401
202402
202403
202404
202405
202406
202407
202408
202409
Completed 202410 of total 269381
202410
202411
202412
202413
202414
202415
202416
202417
202418
202419
Completed 202420 of total 269381
202420
202421
202422
202423
202424
202425
202426
202427
202428
202429
Completed 202430 of total 269381
202430
202431
202432
202433
202434
202435
202436
202437
202438
202439
Completed 202440 of total 269381
202440
202441
202442
202443
202444
202445
202446
202447
202448
202449
Completed 202450 of total 269381
202450
202451
202452
202453
202454
202455
202456
202457
202458
202459
Completed 202460 of total 269381
202460
202461
202462
202463
202464
202465
202466
202467
202468
202469
Completed 202470 of total 269381
202470
202471
202472
202473
202474
202475
202476
202477
202478
202479
Completed 202480 of total 269381
202480
202481
202482
202483
202484
20248

203184
203185
203186
203187
203188
203189
Completed 203190 of total 269381
203190
203191
203192
203193
203194
203195
203196
203197
203198
203199
Completed 203200 of total 269381
203200
203201
203202
203203
203204
203205
203206
203207
203208
203209
Completed 203210 of total 269381
203210
203211
203212
203213
203214
203215
203216
203217
203218
203219
Completed 203220 of total 269381
203220
203221
203222
203223
203224
203225
203226
203227
203228
203229
Completed 203230 of total 269381
203230
203231
203232
203233
203234
203235
203236
203237
203238
203239
Completed 203240 of total 269381
203240
203241
203242
203243
203244
203245
203246
203247
203248
203249
Completed 203250 of total 269381
203250
203251
203252
203253
203254
203255
203256
203257
203258
203259
Completed 203260 of total 269381
203260
203261
203262
203263
203264
203265
203266
203267
203268
203269
Completed 203270 of total 269381
203270
203271
203272
203273
203274
203275
203276
203277
203278
203279
Completed 203280 of total 26938

...

226289
Completed 226290 of total 269381
226290
226291
226292
226293
226294
226295
226296
226297
226298
226299
Completed 226300 of total 269381
226300
226301
226302
226303
226304
226305
226306
226307
226308
226309
Completed 226310 of total 269381
226310
226311
226312
226313
226314
226315
226316
226317
226318
226319
Completed 226320 of total 269381
226320
226321
226322
226323
226324
226325
226326
226327
226328
226329
Completed 226330 of total 269381
226330
226331
226332
226333
226334
226335
226336
226337
226338
226339
Completed 226340 of total 269381
226340
226341
226342
226343
226344
226345
226346
226347
226348
226349
Completed 226350 of total 269381
226350
226351
226352
226353
226354
226355
226356
226357
226358
226359
Completed 226360 of total 269381
226360
226361
226362
226363
226364
226365
226366
226367
226368
226369
Completed 226370 of total 269381
226370
226371
226372
226373
226374
226375
226376
226377
226378
226379
Completed 226380 of total 269381
226380
226381
226382
226383
22638

227083
227084
227085
227086
227087
227088
227089
Completed 227090 of total 269381
227090
227091
227092
227093
227094
227095
227096
227097
227098
227099
Completed 227100 of total 269381
227100
227101
227102
227103
227104
227105
227106
227107
227108
227109
Completed 227110 of total 269381
227110
227111
227112
227113
227114
227115
227116
227117
227118
227119
Completed 227120 of total 269381
227120
227121
227122
227123
227124
227125
227126
227127
227128
227129
Completed 227130 of total 269381
227130
227131
227132
227133
227134
227135
227136
227137
227138
227139
Completed 227140 of total 269381
227140
227141
227142
227143
227144
227145
227146
227147
227148
227149
Completed 227150 of total 269381
227150
227151
227152
227153
227154
227155
227156
227157
227158
227159
Completed 227160 of total 269381
227160
227161
227162
227163
227164
227165
227166
227167
227168
227169
Completed 227170 of total 269381
227170
227171
227172
227173
227174
227175
227176
227177
227178
227179
Completed 227180 of tota

227881
227882
227883
227884
227885
227886
227887
227888
227889
Completed 227890 of total 269381
227890
227891
227892
227893
227894
227895
227896
227897
227898
227899
Completed 227900 of total 269381
227900
227901
227902
227903
227904
227905
227906
227907
227908
227909
Completed 227910 of total 269381
227910
227911
227912
227913
227914
227915
227916
227917
227918
227919
Completed 227920 of total 269381
227920
227921
227922
227923
227924
227925
227926
227927
227928
227929
Completed 227930 of total 269381
227930
227931
227932
227933
227934
227935
227936
227937
227938
227939
Completed 227940 of total 269381
227940
227941
227942
227943
227944
227945
227946
227947
227948
227949
Completed 227950 of total 269381
227950
227951
227952
227953
227954
227955
227956
227957
227958
227959
Completed 227960 of total 269381
227960
227961
227962
227963
227964
227965
227966
227967
227968
227969
Completed 227970 of total 269381
227970
227971
227972
227973
227974
227975
227976
227977
227978
227979
Completed 

228679
Completed 228680 of total 269381
228680
228681
228682
228683
228684
228685
228686
228687
228688
228689
Completed 228690 of total 269381
228690
228691
228692
228693
228694
228695
228696
228697
228698
228699
Completed 228700 of total 269381
228700
228701
228702
228703
228704
228705
228706
228707
228708
228709
Completed 228710 of total 269381
228710
228711
228712
228713
228714
228715
228716
228717
228718
228719
Completed 228720 of total 269381
228720
228721
228722
228723
228724
228725
228726
228727
228728
228729
Completed 228730 of total 269381
228730
228731
228732
228733
228734
228735
228736
228737
228738
228739
Completed 228740 of total 269381
228740
228741
228742
228743
228744
228745
228746
228747
228748
228749
Completed 228750 of total 269381
228750
228751
228752
228753
228754
228755
228756
228757
228758
228759
Completed 228760 of total 269381
228760
228761
228762
228763
228764
228765
228766
228767
228768
228769
Completed 228770 of total 269381
228770
228771
228772
228773
22877

229473
229474
229475
229476
229477
229478
229479
Completed 229480 of total 269381
229480
229481
229482
229483
229484
229485
229486
229487
229488
229489
Completed 229490 of total 269381
229490
229491
229492
229493
229494
229495
229496
229497
229498
229499
Completed 229500 of total 269381
229500
229501
229502
229503
229504
229505
229506
229507
229508
229509
Completed 229510 of total 269381
229510
229511
229512
229513
229514
229515
229516
229517
229518
229519
Completed 229520 of total 269381
229520
229521
229522
229523
229524
229525
229526
229527
229528
229529
Completed 229530 of total 269381
229530
229531
229532
229533
229534
229535
229536
229537
229538
229539
Completed 229540 of total 269381
229540
229541
229542
229543
229544
229545
229546
229547
229548
229549
Completed 229550 of total 269381
229550
229551
229552
229553
229554
229555
229556
229557
229558
229559
Completed 229560 of total 269381
229560
229561
229562
229563
229564
229565
229566
229567
229568
229569
Completed 229570 of tota